### Walkthrough of CT image processing steps
by: Sparkle Russell-Puleri

The goal of this high-level notebook is to walkthrough some quick image pre-processing steps that can benefit from the Rust pandas/polars extensions that we are proposing:
1. Loading operations
2. Metadata Parsing
3. High level image standardization:
   - Convert from pixels to HU 
   - Resample
   - Convert to nrrd, or other storage types, can be:
     - 2D (flatten the Dicom structure)
     - 3D keep structure 


**Disclaimer: This is my approach to image processing, and would not represent all researchers in this space

In [2]:
import requests
import numpy as np
import pandas as pd
import pydicom 
import pylibjpeg
import tcia_utils

### Downloading Medical Images for testing from the TCIA site 
In this example, I used the NSLC Radiogenomics dataset

DICOM Structure essentials: [DICOM Standard](https://www.google.com/url?sa=i&url=https%3A%2F%2Fdicom.innolitics.com%2Fciods%2Frt-dose%2Fimage-plane%2F00280030&psig=AOvVaw0RcFQSHqaMiCJbxjzmAooG&ust=1693778864629000&source=images&cd=vfe&opi=89978449&ved=0CBEQjhxqFwoTCPDpkI74jIEDFQAAAAAdAAAAABAE)

In [3]:
# get list of available collections as JSON -> Medical Images in DICOM format from the TCIA site
tcia_utils.nbia.getCollections()

2023-09-03 11:06:00,119:INFO:Calling... https://services.cancerimagingarchive.net/nbia-api/services/v1/getCollectionValues with parameters {}


[{'Collection': '4D-Lung'},
 {'Collection': 'ACRIN-6698'},
 {'Collection': 'ACRIN-Contralateral-Breast-MR'},
 {'Collection': 'ACRIN-FLT-Breast'},
 {'Collection': 'ACRIN-NSCLC-FDG-PET'},
 {'Collection': 'Adrenal-ACC-Ki67-Seg'},
 {'Collection': 'Anti-PD-1_Lung'},
 {'Collection': 'B-mode-and-CEUS-Liver'},
 {'Collection': 'BREAST-DIAGNOSIS'},
 {'Collection': 'Breast-Cancer-Screening-DBT'},
 {'Collection': 'Breast-MRI-NACT-Pilot'},
 {'Collection': 'C4KC-KiTS'},
 {'Collection': 'CBIS-DDSM'},
 {'Collection': 'CC-Radiomics-Phantom'},
 {'Collection': 'CC-Radiomics-Phantom-2'},
 {'Collection': 'CC-Radiomics-Phantom-3'},
 {'Collection': 'CC-Tumor-Heterogeneity'},
 {'Collection': 'CMB-CRC'},
 {'Collection': 'CMB-GEC'},
 {'Collection': 'CMB-LCA'},
 {'Collection': 'CMB-MEL'},
 {'Collection': 'CMB-MML'},
 {'Collection': 'CMB-PCA'},
 {'Collection': 'CMMD'},
 {'Collection': 'COVID-19-AR'},
 {'Collection': 'COVID-19-NY-SBU'},
 {'Collection': 'CPTAC-CCRCC'},
 {'Collection': 'CPTAC-CM'},
 {'Collection': '

In [ ]:
data = tcia_utils.nbia.getSeries(collection = "NSCLC Radiogenomics")
print(data)

2023-09-02 14:14:39,536:INFO:Calling... https://services.cancerimagingarchive.net/nbia-api/services/v1/getSeries with parameters {'Collection': 'NSCLC Radiogenomics'}


[{'SeriesInstanceUID': '1.3.6.1.4.1.14519.5.2.1.4334.1501.447728746190490431768087181101', 'StudyInstanceUID': '1.3.6.1.4.1.14519.5.2.1.4334.1501.162279367121844086709629964588', 'Modality': 'CT', 'ProtocolName': '5.1 CT CHEST ROUTINE', 'SeriesDate': '1990-10-16 00:00:00.0', 'SeriesDescription': 'Recon 3: CT CHEST W/O', 'SeriesNumber': 4, 'Collection': 'NSCLC Radiogenomics', 'PatientID': 'R01-016', 'Manufacturer': 'GE MEDICAL SYSTEMS', 'ManufacturerModelName': 'LightSpeed16', 'SoftwareVersions': 'LightSpeedverrel', 'ImageCount': 277, 'TimeStamp': '2017-12-12 12:58:26.0', 'LicenseName': 'Creative Commons Attribution 3.0 Unported License', 'LicenseURI': 'http://creativecommons.org/licenses/by/3.0/', 'CollectionURI': 'https://doi.org/10.7937/K9/TCIA.2017.7hs46erv', 'FileSize': 146022934}, {'SeriesInstanceUID': '1.3.6.1.4.1.14519.5.2.1.4334.1501.322979353264523657170838529817', 'StudyInstanceUID': '1.3.6.1.4.1.14519.5.2.1.4334.1501.756412277977427280636198129778', 'Modality': 'CT', 'Protoc

In [4]:
# tcia_utils.nbia.downloadSeries(data, number = 3)

"""
2023-09-02 14:15:15,643:INFO:Downloading 3 out of 1351 Series Instance UIDs (scans).
2023-09-02 14:15:15,643:INFO:Downloading... https://services.cancerimagingarchive.net/nbia-api/services/v1/getImage?NewFileNames=Yes&SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.4334.1501.447728746190490431768087181101
2023-09-02 14:15:32,431:INFO:Downloading... https://services.cancerimagingarchive.net/nbia-api/services/v1/getImage?NewFileNames=Yes&SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.4334.1501.322979353264523657170838529817
2023-09-02 14:15:48,706:INFO:Downloading... https://services.cancerimagingarchive.net/nbia-api/services/v1/getImage?NewFileNames=Yes&SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.4334.1501.240772414306783411390403639800
2023-09-02 14:16:10,625:INFO:Downloaded 3 out of 3 requested series from a total of 1351 Series Instance UIDs (scans).
0 failed to download.
0 previously downloaded.
"""

'\n2023-09-02 14:15:15,643:INFO:Downloading 3 out of 1351 Series Instance UIDs (scans).\n2023-09-02 14:15:15,643:INFO:Downloading... https://services.cancerimagingarchive.net/nbia-api/services/v1/getImage?NewFileNames=Yes&SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.4334.1501.447728746190490431768087181101\n2023-09-02 14:15:32,431:INFO:Downloading... https://services.cancerimagingarchive.net/nbia-api/services/v1/getImage?NewFileNames=Yes&SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.4334.1501.322979353264523657170838529817\n2023-09-02 14:15:48,706:INFO:Downloading... https://services.cancerimagingarchive.net/nbia-api/services/v1/getImage?NewFileNames=Yes&SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.4334.1501.240772414306783411390403639800\n2023-09-02 14:16:10,625:INFO:Downloaded 3 out of 3 requested series from a total of 1351 Series Instance UIDs (scans).\n0 failed to download.\n0 previously downloaded.\n'

### Step 1: File system structure
In the medical imaging world, we would typically group patients into one directory and within that directory the patient can have multiple visits in sub folders or multiple scans within one visit (this can mean that some of the images were taken because the physician wanted a different view or the first scan was not ideal). Here is a typical structure:

**At the Data Level:**
```
(test) sparklerussell@Sparkles-MacBook-Air data % tree -x -L 2
.
└── Data
    ├── Patient_1
    ├── Patient_2
    └── Patient_3
```

**At the Patient Level:**
```
(test) sparklerussell@Sparkles-MacBook-Air data/Patient_1 % tree -x -L 2
.
└── Patient_1
    ├── Visit_1
    ├── Visit_2
    └── Visit_3
```
**At the Visit Level:**
```
(test) sparklerussell@Sparkles-MacBook-Air data/Patient_1/ % tree -x -L 2
.
└── Visit_1
    ├── Scan_1
    ├── Scan_2
    └── Scan_3

```
**At the Scan Level:** Can be a dicom volume (intact) to a flattened into individual dicom slices like the current dataset we are working with:

```
(test) sparklerussell@Sparkles-MacBook-Air EXAMPLE % tree -x -L 2
.
└── Scan_1
    ├── DICOM_volume
    ├── Scan_1
    └── Scan_2
```


#### Rust API will need to: CT Imaging use case
1. Walk the directory to find all of the `.dcm` files and store their location

### Step 2: Loading Operations

In [4]:
path = "data/tciaDownload/pat1_1.3.6.1.4.1.14519.5.2.1.4334.1501.447728746190490431768087181101/1-001.dcm"

data = pydicom.dcmread(path)

In [48]:

tags = []
for tag in data:
    tags.append(tag.name)

## Here we can see there are 85 tags (some are nested so there will be more!!!!)
tags


['Specific Character Set',
 'Image Type',
 'Instance Creation Date',
 'Instance Creation Time',
 'SOP Class UID',
 'SOP Instance UID',
 'Study Date',
 'Series Date',
 'Acquisition Date',
 'Content Date',
 'Study Time',
 'Series Time',
 'Acquisition Time',
 'Content Time',
 'Accession Number',
 'Modality',
 'Manufacturer',
 "Referring Physician's Name",
 'Study Description',
 'Procedure Code Sequence',
 'Series Description',
 "Manufacturer's Model Name",
 'Referenced Image Sequence',
 "Patient's Name",
 'Patient ID',
 "Patient's Birth Date",
 "Patient's Sex",
 "Patient's Age",
 'Patient Identity Removed',
 'De-identification Method',
 'De-identification Method Code Sequence',
 'Private Creator',
 'Private tag data',
 'Private tag data',
 'Scan Options',
 'Slice Thickness',
 'KVP',
 'Data Collection Diameter',
 'Software Versions',
 'Protocol Name',
 'Reconstruction Diameter',
 'Distance Source to Detector',
 'Distance Source to Patient',
 'Gantry/Detector Tilt',
 'Table Height',
 'Rotat

In [35]:
## Here you can see an example of what a nested tag: (0012, 0064)  De-identification Method Code Sequence  8 item(s) ---- > nested with 8 items
data

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 196
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.6.1.4.1.14519.5.2.1.4334.1501.108563624120950452032304933862
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.40.0.13.1.1.1
(0002, 0013) Implementation Version Name         SH: 'dcm4che-1.4.35'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL']
(0008, 0012) Instance Creation Date              DA: '19901016'
(0008, 0013) Instance Creation Time              TM: '143231.000000'
(0008, 0016) SOP Class UID                       UI: CT Image Storage
(0008, 0018) SO

#### Rust API Ideas:
1. pandas.read_dcm(&path).parse_meta() --> parse the metadata and store in a pyarrow data structure
  
   **Note:**For the extraction of the metadata we can focus on only high level tags (no nesting) and keep the others as is as other researchers may have a reason to use this

### Step 3: Metadata exploration and initial pre-processing
Explore the metadata to better understand image acquistion settings.

##### How were the images encoded? Would need to check that the current dicom-rs package supports the images

In [47]:

## In this  dataset it is: Explitcit Enidian --> [Source](https://pydicom.github.io/pydicom/stable/old/image_data_handlers.html#supported-transfer-syntaxes)
data.file_meta.TransferSyntaxUID ## Rust dicom-rs will handle this...

'1.2.840.10008.1.2.1'

##### Image bit allocation and storage 

In [39]:
## Here we can see that the image is signed 16-bit (from the raw pixel values) --> will need an option to convert to others depending on ML/analysis needs 
# (can also check the Bits Allocated & Stored tags)
data.pixel_array.max(), data.pixel_array.min()

(2489, -1024)

##### Check pixel spacing and thickness:

In [52]:
# Check pixel spacing --> 0.82mm in the x-direction, 0.82mm in the y-direction, and thickness --> 1.25 mm z-direction the slice thickness is needed
# for downstream resampling.
data.PixelSpacing, data.SliceThickness

([0.820312, 0.820312], '1.25')

#### Rust API use cases: Image preprocessing (pandas or polars) basic high level steps
1. pandas.read_dicom.apply(convert_hu) --> apply the all extracted pixel array data to convert back from pixels to radiodensity units for ML
2. resample = resample(image, converted_hu_image, new_spacing)  --> pandas.read_dicom.apply(resample) --> resample pixels 
3. Convert file to nrrd, npy/z-->  final_data --> pandas.apply(convert_x_type)

### Other functions to help with cohort creation or exploration:
1. pandas.read_dicom.filter(col('SliceThichkness').0 < 1)  ->  SliceThickness is a list of z values --> '1.25'
2. pandas.read_dicom.filter(col('PixelSpacing').0 < 1)  --users may want to count how many images have a pixel spacing less than 1 in the x or y direction

## Rust API use cases by task:
**File System i/o:**
1. Walk the directory to find all of the `.dcm` files and store their location --> pandas.read_dcm(&path) --> in this function we should be able to do most of the other extraction tasks

**Loading:**
1. pandas.read_dcm(&path).parse_meta() --> parse the metadata and store in a pyarrow data structure

**Metadata exploration:**
1. pandas.read_dicom.apply(convert_hu) --> apply the all extracted pixel array data to convert back from pixels to radiodensity units for ML
2. resample = resample(image, converted_hu_image, new_spacing)  --> pandas.read_dicom.apply(resample) --> resample pixels 
3. Convert file to nrrd, npy/z-->  final_data --> pandas.apply(convert_x_type)

##### Other functions to help with cohort creation or exploration:
1. pandas.read_dicom.filter(col('SliceThichkness').0 < 1)  ->  SliceThickness is a list of z values --> '1.25'
2. pandas.read_dicom.filter(col('PixelSpacing').0 < 1)  --users may want to count how many images have a pixel spacing less than 1 in the x or y direction
